In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

In [2]:
(train_dataset,test_dataset),ds_info = tfds.load('imdb_reviews',
                                                 with_info = True,
                                         split = ['train','test'],
                                                  as_supervised = True,
                                         shuffle_files = True,
                                         batch_size = 128

                                                 )

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5EO6JY/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5EO6JY/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5EO6JY/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
def text_vectorization(dataset,vectorizer):
  """
  Uses TextVectorization Layer outside of the model.
  The purpose is to make the NN model suitable to .h5 format.

  Parameters
  ----------
  dataset: Tensorflow dataset containing data and labels.
  vectorizer: A TextVectorization layer adapted to train_dataset
  """
  #conduct all the process in CPU to CPU-GPU conflict
  with tf.device("/cpu:0"):
    outputs = []
    label_list = []

    #tokenize and pad the data and store it
    for x, y in dataset:
        output = vectorizer(x)
        outputs.append(output)
        label_list.append(y)

    #concatenate the labels and data
    X_vectorized = tf.concat(outputs, axis=0)
    Y_labels = tf.concat(label_list, axis=0)

    vectorized = tf.data.Dataset.from_tensor_slices((X_vectorized, Y_labels)).batch(32).prefetch(tf.data.AUTOTUNE)
    return vectorized

def define_callbacks(model):
  es = tf.keras.callbacks.EarlyStopping(patience = 5,verbose = 1, restore_best_weights = True)
  mc = tf.keras.callbacks.ModelCheckpoint(filepath = f"./ModelCheckpoints/{model.name}.ckpt",
                                         save_best_only = True,
                                         save_weights_only = True)
  tb = tf.keras.callbacks.TensorBoard(log_dir = f"./TensorboardLogs/{model.name}")

  return es,mc,tb

In [4]:
#get the vocab length
vectorizer = layers.TextVectorization(max_tokens = None)
vectorizer.adapt(train_dataset.map(lambda x,y: x))
total_words = vectorizer.vocabulary_size()

In [5]:
#save the length of the sequences to find the output sequence length
text_lengths = []
train_dataset_unbatched = train_dataset.unbatch()
for text,label in train_dataset_unbatched: #unbatch the dataset
  text_lengths.append(len(text.numpy().split())) #append the sentence length

In [6]:
MAX_TOKENS = 20000
OUTPUT_SEQUENCE_LENGTH = int(tfp.stats.percentile(text_lengths,95).numpy())
EMBEDDING_DIM = 128
MODEL_NAME = "model1"

#define the actual vectorizer
vectorizer = tf.keras.layers.TextVectorization(max_tokens = MAX_TOKENS,
                                               output_sequence_length = OUTPUT_SEQUENCE_LENGTH)

vectorizer.adapt(train_dataset.map(lambda x,y: x))
#preprocess the data
train_dataset_vectorized = text_vectorization(train_dataset,vectorizer)
test_dataset_vectorized = text_vectorization(test_dataset,vectorizer)


inputs = tf.keras.layers.Input(shape = (598,))

x = layers.Embedding(input_dim = MAX_TOKENS,
                      output_dim = EMBEDDING_DIM)(inputs)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

#create the base model
model = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#create the callbacks
es,mc,tb = define_callbacks(model)

#compile the model
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#fit the model
history = model.fit(train_dataset_vectorized,validation_data = test_dataset_vectorized,epochs = 20,
                    callbacks = [es,mc,tb])



Epoch 1/20
782/782 [==============================] - 114s 140ms/step - loss: 0.6009 - accuracy: 0.7174 - val_loss: 0.4819 - val_accuracy: 0.8200
Epoch 2/20
782/782 [==============================] - 6s 8ms/step - loss: 0.3840 - accuracy: 0.8654 - val_loss: 0.3573 - val_accuracy: 0.8666
Epoch 3/20
782/782 [==============================] - 5s 7ms/step - loss: 0.2899 - accuracy: 0.8964 - val_loss: 0.3140 - val_accuracy: 0.8808
Epoch 4/20
782/782 [==============================] - 6s 8ms/step - loss: 0.2404 - accuracy: 0.9136 - val_loss: 0.2945 - val_accuracy: 0.8857
Epoch 5/20
782/782 [==============================] - 7s 8ms/step - loss: 0.2061 - accuracy: 0.9276 - val_loss: 0.2858 - val_accuracy: 0.8881
Epoch 6/20
782/782 [==============================] - 5s 6ms/step - loss: 0.1794 - accuracy: 0.9376 - val_loss: 0.2837 - val_accuracy: 0.8886
Epoch 7/20
782/782 [==============================] - 7s 10ms/step - loss: 0.1574 - accuracy: 0.9470 - val_loss: 0.2865 - val_accuracy: 0.8879
E

In [8]:
MAX_TOKENS = 20000
OUTPUT_SEQUENCE_LENGTH = int(tfp.stats.percentile(text_lengths,95).numpy())
EMBEDDING_DIM = 128
MODEL_NAME = "model2"

#define the actual vectorizer
vectorizer = tf.keras.layers.TextVectorization(max_tokens = MAX_TOKENS,
                                               output_sequence_length = OUTPUT_SEQUENCE_LENGTH)
vectorizer.adapt(train_dataset.map(lambda x,y: x))
#preprocess the data
train_dataset_vectorized = text_vectorization(train_dataset,vectorizer)
test_dataset_vectorized = text_vectorization(test_dataset,vectorizer)


inputs = tf.keras.layers.Input(shape = (598,))

x = layers.Embedding(input_dim = MAX_TOKENS,
                      output_dim = EMBEDDING_DIM)(inputs)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64,activation = 'relu')(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

#create the base model
model2 = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#create the callbacks
es,mc,tb = define_callbacks(model2)

#compile the model
model2.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#fit the model
history2 = model2.fit(train_dataset_vectorized,validation_data = test_dataset_vectorized,epochs = 20,
                    callbacks = [es,mc,tb])


Epoch 1/20
782/782 [==============================] - 94s 118ms/step - loss: 0.4345 - accuracy: 0.7961 - val_loss: 0.2963 - val_accuracy: 0.8828
Epoch 2/20
782/782 [==============================] - 8s 10ms/step - loss: 0.2172 - accuracy: 0.9151 - val_loss: 0.2835 - val_accuracy: 0.8856
Epoch 3/20
782/782 [==============================] - 5s 6ms/step - loss: 0.1546 - accuracy: 0.9430 - val_loss: 0.3003 - val_accuracy: 0.8840
Epoch 4/20
782/782 [==============================] - 7s 9ms/step - loss: 0.1157 - accuracy: 0.9593 - val_loss: 0.3444 - val_accuracy: 0.8789
Epoch 5/20
782/782 [==============================] - 7s 8ms/step - loss: 0.0942 - accuracy: 0.9692 - val_loss: 0.4030 - val_accuracy: 0.8715
Epoch 6/20
782/782 [==============================] - 6s 7ms/step - loss: 0.0848 - accuracy: 0.9703 - val_loss: 0.5479 - val_accuracy: 0.8436
Epoch 7/20
782/782 [==============================] - 6s 8ms/step - loss: 0.0835 - accuracy: 0.9685 - val_loss: 0.4693 - val_accuracy: 0.8661
Ep

In [21]:
MAX_TOKENS = 10000
OUTPUT_SEQUENCE_LENGTH = int(tfp.stats.percentile(text_lengths,95).numpy())
EMBEDDING_DIM = 50
MODEL_NAME = 'model3'

#define the actual vectorizer
vectorizer = tf.keras.layers.TextVectorization(max_tokens = MAX_TOKENS,
                                               output_sequence_length = OUTPUT_SEQUENCE_LENGTH)
vectorizer.adapt(train_dataset.map(lambda x,y: x))

#preprocess the data
train_dataset_vectorized = text_vectorization(train_dataset,vectorizer)
test_dataset_vectorized = text_vectorization(test_dataset,vectorizer)

inputs = tf.keras.layers.Input(shape = (598,))
x = layers.Embedding(input_dim = MAX_TOKENS,
                      output_dim = EMBEDDING_DIM)(inputs)
x = layers.GlobalAveragePooling1D()(x)
#x = layers.Dense(64,activation = 'relu')(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

#create the base model
model3 = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#create the callbacks
es,mc,tb = define_callbacks(model3)

#compile the model
model3.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#fit the model
history3 = model3.fit(train_dataset_vectorized,validation_data = test_dataset_vectorized,epochs = 20,
                    callbacks = [es,mc,tb])

Epoch 1/20
782/782 [==============================] - 114s 145ms/step - loss: 0.6417 - accuracy: 0.6888 - val_loss: 0.5626 - val_accuracy: 0.7737
Epoch 2/20
782/782 [==============================] - 11s 14ms/step - loss: 0.4714 - accuracy: 0.8340 - val_loss: 0.4242 - val_accuracy: 0.8458
Epoch 3/20
782/782 [==============================] - 6s 8ms/step - loss: 0.3634 - accuracy: 0.8724 - val_loss: 0.3597 - val_accuracy: 0.8654
Epoch 4/20
782/782 [==============================] - 9s 11ms/step - loss: 0.3079 - accuracy: 0.8880 - val_loss: 0.3271 - val_accuracy: 0.8751
Epoch 5/20
782/782 [==============================] - 5s 6ms/step - loss: 0.2732 - accuracy: 0.9001 - val_loss: 0.3084 - val_accuracy: 0.8811
Epoch 6/20
782/782 [==============================] - 6s 8ms/step - loss: 0.2479 - accuracy: 0.9099 - val_loss: 0.2969 - val_accuracy: 0.8832
Epoch 7/20
782/782 [==============================] - 7s 9ms/step - loss: 0.2279 - accuracy: 0.9172 - val_loss: 0.2901 - val_accuracy: 0.8851

In [22]:
MAX_TOKENS = 10000
OUTPUT_SEQUENCE_LENGTH = int(tfp.stats.percentile(text_lengths,95).numpy())
EMBEDDING_DIM = 50
MODEL_NAME = 'model4'


inputs = tf.keras.layers.Input(shape = (598,))
x = layers.Embedding(input_dim = MAX_TOKENS,
                      output_dim = EMBEDDING_DIM)(inputs)
x = layers.Flatten()(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

#create the base model
model4 = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#create the callbacks
es,mc,tb = define_callbacks(model4)

#compile the model
model4.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#fit the model
history4 = model4.fit(train_dataset_vectorized,validation_data = test_dataset_vectorized,epochs = 20,
                    callbacks = [es,mc,tb])



Epoch 1/20
782/782 [==============================] - 77s 98ms/step - loss: 0.4363 - accuracy: 0.7901 - val_loss: 0.3087 - val_accuracy: 0.8720
Epoch 2/20
782/782 [==============================] - 5s 6ms/step - loss: 0.1882 - accuracy: 0.9317 - val_loss: 0.3163 - val_accuracy: 0.8675
Epoch 3/20
782/782 [==============================] - 5s 7ms/step - loss: 0.0859 - accuracy: 0.9788 - val_loss: 0.3537 - val_accuracy: 0.8626
Epoch 4/20
782/782 [==============================] - 6s 7ms/step - loss: 0.0330 - accuracy: 0.9958 - val_loss: 0.3936 - val_accuracy: 0.8591
Epoch 5/20
782/782 [==============================] - 5s 7ms/step - loss: 0.0131 - accuracy: 0.9992 - val_loss: 0.4278 - val_accuracy: 0.8601
Epoch 6/20
782/782 [==============================] - 6s 7ms/step - loss: 0.0058 - accuracy: 1.0000 - val_loss: 0.4639 - val_accuracy: 0.8580
Epoch 6: early stopping


In [23]:
!tensorboard dev upload --logdir /content/TensorboardLogs

2023-10-01 06:49:33.421879: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

/content/TensorboardLogs

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

To sign in with the TensorBoard uploader:

1. On your computer or phone, visit:

   https://www.google.com/device

2. Sign in with your Google account, then enter:

   NHCS-QFSL


Upload started and will continue reading any new data as it's added to t

Model1 is the best one

**ÇOK ÖNEMLİ** TEXT VECTORIZATION H5'E DAHİL EDİLEMEZ.

In [25]:
#clone the model
final_model = tf.keras.models.clone_model(model)

#load the best weights
final_model.load_weights("/content/ModelCheckpoints/model1.ckpt")

#compile the model
final_model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

#evaluate the test dataset
final_model.evaluate(test_dataset_vectorized)

782/782 [==============================] - 2s 2ms/step - loss: 0.2855 - accuracy: 0.8860


[0.28552237153053284, 0.8859999775886536]

In [26]:
!tensorboard dev list

2023-10-01 06:55:16.778509: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
https://tensorboard.dev/experiment/yGLy3KODTamIar5z7XTjkg/
	Name                 [No Name]
	Description          [No Description]
	Id                   yGLy3KODTamIar5z7XTjkg
	Created              2023-10-01 06:50:01 (5 minutes ago)
	Updated              2023-10-01 06:50:05 (5 minutes ago)
	Runs                 8
	Tags                 5
	Scalars              228
	Tensor bytes         0
	Binary object bytes  231713
https://tensorboard.dev/experiment/gyy8IjoPTyOm7cx9EyE2rw/
	Name                 [No Name]
	Description          [No Description]
	Id                   gyy8IjoPTyOm7cx9EyE2rw
	Created              2023-09-29 08:49:11
	Updated              2023-09-29 08:49:16
	Runs                 8
	Tags                 5
	Scalars              228
	Tensor bytes         0
	Binary object bytes  279251
https://tensorboard.dev/experiment/TXGkSngLSkSRdi6qqoKIvQ/
	Name        

In [29]:
!tensorboard dev delete --experiment_id TXGkSngLSkSRdi6qqoKIvQ

2023-10-01 06:56:11.041203: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Deleted experiment TXGkSngLSkSRdi6qqoKIvQ.
